In [4]:
'''
Name : Himanshu
Roll No: 16MA20020

Assignment 1b
'''

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df= pd.read_csv('Iris_Data.csv')
df.head()

'''
You will not import any other library other than these provided.

We provide the iris_dataset for the classification task
There are 4 dependent variables columns(1-4).
The last column (category of the flower) is what we wish to predict

The first part of this task is similar to assignment 1 a
'''
# reads the file and stores in 2 numpy arrays.
# X has the input features and Y has the output value in numpy array

X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

rows,cols= X.shape[0], X.shape[1] #?, #? 
# how to get the number of rows and columns in the dataset.
# Rows correspond to the number of input instances, columns correspond to the feature of an input

print(rows,cols)

np.random.seed(42) # to ensure that the same seed is generated
print(X[0])
# write code to shuffle the dataset

def shuffle_dataset(X,Y):
    
    '''
        Write code to shuffle the dataset here. 
        
        Args: 
            X: Input feature ndarray
            Y: Input values ndarray
            
        Return:
            X and Y shuffled in place
    
    '''
    arr = np.arange(rows)
    np.random.shuffle(arr)
    X = X[arr]
    Y = Y[arr]
    
    return [X, Y]
    
X,Y = shuffle_dataset(X,Y)
print(X[0])
training_size = int(0.8*rows)
X_train = np.nan_to_num(X[:training_size])
y_train = np.nan_to_num(Y[:training_size])
X_test = np.nan_to_num(X[training_size:])
y_test = np.nan_to_num(Y[training_size:])

150 4
[5.1 3.5 1.4 0.2]
[6.1 2.8 4.7 1.2]


In [5]:
X_train[0]

array([6.1, 2.8, 4.7, 1.2])

## Defining the Feed Forward Neural Network class

In [15]:
class NeuralNetwork(object):
    
    def __init__(self, input_no, hidden_no, output_no ):
        '''
            Initialize the Neural network model 
            Args:
                input_no : no of input features (no of cols)
                hidden_no: no of hidden nodes in the model
                output_no: no of categories our model can classify
                      
        
        '''
        self.h= np.zeros(hidden_no) # Initialize the hidden layer with zero ?
        self.w1= np.random.rand(hidden_no,cols) # Initialize the weights from the input to the hidden layer uniformly with values between 0 and 0.01  ?
        #print(self.w1.shape)
        self.b1= np.random.rand(hidden_no) # Initialize the biases uniformly with values between 0 and 0.01 equal to the number of hidden nodes  ?
        self.w2= np.random.rand(output_no,hidden_no) # Initialize the weights from the hidden layer to the output uniformly with values between 0 and 0.01 ? 
        self.b2= np.random.rand(output_no)# Initialize the biases uniformly with values between 0 and 0.01 equal to the number of output categories ?

    
    def forward(self, x):
        '''
            Do a forward pass on the NN model 
            Args: 
                x : Input feature matrix 
                
            Return:
                y_pred : list of predicted probabilities of x

                h = relu(w1.x+b1) 
                y_pred = softmax(w2.h+b2)
                
        
        '''
        y_pred=[]
        
        # evaluate activations of first hidden layer
        self.h = np.matmul(x,np.transpose(self.w1)) + self.b1
        for i in range(self.h.shape[0]):
            for j in range(self.h.shape[1]):
                if self.h[i][j]<0:
                    self.h[i][j]=0
        
        y_pred = np.matmul(self.h, np.transpose(self.w2)) + self.b2
        sum_ = np.sum(np.exp(y_pred), axis=1)
        #print(sum_.shape)
        y_pred = np.divide(np.exp(y_pred), (sum_.reshape(-1,1)))
        
        return y_pred

    
    def backward(self, x, y_train, y_pred, lr):
        
        '''
            Do a backward pass on the NN model. 
            Computes all gradients and updates the parameters w1, b1, w2, b2
            
            Args:
                x: input matrix X 
                y_train: actual category of the feature/ data point
                y_pred: predicted probabilities of the categories obtained during forward pass
                lr: learning rate        
        '''
        
        y_ = np.zeros((y_train.shape[0], 3))
        for i in range(y_.shape[0]):
            y_[y_train[i]]=1
        y_train = y_
        
        #print(y_train[0])
        
        H_in = np.matmul(x,np.transpose(self.w1)) + self.b1
        h = H_in
        for i in range(h.shape[0]):
            for j in range(h.shape[1]):
                if h[i][j]<0:
                    h[i][j]=0
        H_out = h
        O_in = np.matmul(h,np.transpose(self.w2)) + self.b2
        O_out = self.forward(x)
        
        dE_dout = -1 * ( np.multiply(y_train, 1/y_pred) + np.multiply((1-y_train), 1/(1-y_pred)) )
        
        dOout_dOin = O_in
        for i in range(O_in.shape[1]):
            dOout_dOin[:,i] = (O_in[:,i]*(np.sum(O_in, axis=1)-O_in[:,i]))/np.power(np.sum(O_in, axis=1), 2)
        
        
        dW2_1 = np.zeros(self.w2.shape)
        db2_1 = np.zeros(self.b2.shape)
        for i in range(x.shape[0]):
            temp = np.ones(self.w2.shape)
            temp = temp*H_out[i]
            temp = temp*dE_dout[i].reshape(-1,1)
            temp = temp*dOout_dOin[i].reshape(-1,1)
            dW2_1 += temp
            
            db2_1 += dE_dout[i]*dOout_dOin[i]
        
        dW2 = dW2_1/x.shape[0]
        db2 = db2_1/x.shape[0]
        
        
        dHout_dHin = (H_in > 0).astype(int)
        
        temp1 = dE_dout*dOout_dOin
        temp2 = np.ones((x.shape[0], self.w2.shape[1], self.w2.shape[0]))
        #temp2 = temp2*temp1
        for i in range(x.shape[0]):
            temp2[i] = temp2[i]*temp1[i]
            temp2[i] = np.multiply(temp2[i], np.transpose(self.w2))
        temp2 = np.sum(temp2, axis=2)
        
        dEtot_dHout = temp2
        
        dW1_1 = np.zeros(self.w1.shape)
        db1_1 = np.zeros(self.b1.shape)
        
        for i in range(x.shape[0]):
            temp = np.ones(self.w1.shape)
            temp = temp*dHout_dHin[i].reshape(-1,1)
            temp = temp*x[i]
            temp = temp*dEtot_dHout[i].reshape(-1,1)
            dW1_1 += temp
            
            db1_1 += dHout_dHin[i]*dEtot_dHout[i]
            
            
        
        dW1 = dW1_1/x.shape[0]
        db1 = db1_1/x.shape[0]
        
        self.w1 = self.w1-dW1*lr
        self.w2 = self.w2-dW2*lr
        self.b1 = self.b1-db1*lr
        self.b2 = self.b2-db2*lr
        
        #print(self.w2)
        
        
def crossEntropy_loss(y_pred, y_train):
    '''
        Computes the cross entropy loss between the predicted values and the actual values
        
        Args:
            y_pred: predicted probabilities of the categories obtained during forward pass
            y_train: actual category of the feature/ data point
    
    '''
    y_ = np.zeros((y_train.shape[0], 3))
    for i in range(y_.shape[0]):
        y_[y_train[i]]=1
    y_train = y_
        
    res = (-1) * ( (y_train * np.log(y_pred)) + ((1-y_train) * np.log(1-y_pred)) )
    res = np.sum(res)/y_pred.shape[0]
    return res

def accuracy(y_pred,y_train):
    '''
        Computes the accuracy between the predicted values and actual labels
    
        Args:
            y_pred: predicted probabilities of the categories obtained during forward pass
            y_train: actual category of the feature/ data point

    '''
    
    y_pred = np.argmax(y_pred, axis=1)
    res = np.sum((y_pred==y_train).astype(int))/y_pred.shape[0]
    return res
        


In [17]:
# Initialize the neural network model and specify the parameters 

hidden_nodes=5
nnobj= NeuralNetwork(cols,hidden_nodes,3)       
epochs = 1000
learning_rate = 1e-2
loss_history = []
epoch_history = []

# Gradient Descent
for e in range(epochs):
    yPred = nnobj.forward(X_train)
    print(crossEntropy_loss(yPred,y_train))
    nnobj.backward(X_train, y_train,yPred, lr=learning_rate)   #, lmda=lmda)

# yPred = nnobj.forward(X_train)
# print(yPred)
    
# train_loss= crossEntropy_loss(nnobj.forward(X_train), y_train)  #?
# train_accuracy= accuracy(nnobj.forward(X_train), y_train) #?
# test_loss= crossEntropy_loss(nnobj.forward(X_test), y_test)# ?
# test_accuracy= accuracy(nnobj.forward(X_test), y_test) #?
    
# print("Final train_loss "+ str(train_loss))    
# print("Final train_accuracy "+ str(train_accuracy))    
# print("Testloss " + str(test_loss))
# print("Accuracy is "+ str(test_accuracy))

        
            

2.6971880610289536
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


/home/singh/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:155: RuntimeWarning: divide by zero encountered in log
/home/singh/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in multiply
/home/singh/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:82: RuntimeWarning: divide by zero encountered in true_divide
/home/singh/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in multiply
/home/singh/anaconda3/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in greater


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [73]:
y_train[0]

0